In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
%matplotlib inline
import os
from tqdm.notebook import  tqdm
import talib

import  mplfinance as mpf

In [2]:
# 这里导入数据
df = pd.read_csv("../数据获取/聚宽数据/最新数据.csv")
df.columns = ['id','i', 'Date','Open','High','Low','Close','Volume']
df['id2'] =df['id'].shift()
df['i'] = range(len(df))
ids = list(df.loc[df['id'] != df['id2'], 'i']) # 这个是单个股票序号的起始。
ids.append(len(df)-1) # 添加最后1天。
df.head()

,id,i,Date,Open,High,Low,Close,Volume,id2
0,000001.XSHE,0,2018-12-24,8.91,8.96,8.83,8.93,53694517.0,NaN
1,000001.XSHE,1,2018-12-25,8.81,8.94,8.73,8.86,61867885.0,000001.XSHE
2,000001.XSHE,2,2018-12-26,8.87,8.93,8.79,8.82,44415942.0,000001.XSHE
3,000001.XSHE,3,2018-12-27,8.96,9.00,8.80,8.80,65873247.0,000001.XSHE
4,000001.XSHE,4,2018-12-28,8.83,8.97,8.83,8.89,60812019.0,000001.XSHE


In [3]:
list(df.columns)

['id', 'i', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'id2']

In [ ]:
days = 20 # 最多N天的
rises = [[] for _ in range(days)] # 几天的涨幅都在这里边。 
ma_n = [5,10,20,30] # 做这么多多线。
duo_days = []   # 
duo_rises = []
tmp = []
# 前后几天的k线图
before_k = 10
after_k = 10

i_img = 1 # 这个是图片的序号

dest_dir = "均线多头k线图"
if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)
    print("创建文件夹:" + dest_dir)

with tqdm(total=len(ids)) as pbar:
    for j in range(len(ids)-1):
        # 对每一个股票进行遍历
        df2 = df.iloc[ids[j]:ids[j+1],:].copy() # 这里筛选一只股票。
        # 这里要有超过200天的数据
        if len(df2) < 200:
            continue
        # 计算均线
        for m in ma_n:
            df2["Ma{}".format(m)] = talib.SMA(df2['Close'], timeperiod=m)
        # 填充空值,我这里填充0
        df2.fillna(value=0, inplace=True)
        # 这里判断是否是均线多头,我这里要组件表达式
        _tmp = []
        for i in range(len(ma_n)-1):
            _tmp.append("(df2['Ma{}'] > df2['Ma{}'])".format(ma_n[i], ma_n[i+1]))
        _tmp2 = " & ".join(_tmp) 
        # 运行表达式
        df3 = eval("df2[{}].copy()".format(_tmp2))
        # 这里开始统计
        # 这里要看看i所在的列是哪个
        i_index = list(df3.columns).index("i")
        close_index = list(df3.columns).index("Close")
        id_index = list(df3.columns).index("id")
        Date_index = list(df3.columns).index("Date")
        k = 0
        last_i = df3.iat[0, i_index] # 最后一个i的值。
        last_close=df3.iat[0, close_index]
        
        while k < len(df3):
            # 遍历
            if k > 0 and  df3.iat[k, i_index]-df3.iat[k-1, i_index] > 1:
                # 超过1天的，这里要计算天数了
                duo_days.append(df3.iat[k-1, i_index]-last_i+1)
                duo_rises.append((df3.iat[k-1, close_index]-last_close)/last_close*100)
                # 我这里要保存这个均线多头的k线图
                start_i = last_i - before_k  if last_i - before_k >= 0 else 0
                end_i = df3.iat[k-1, i_index] + after_k if df3.iat[k-1, i_index] + after_k < len(df) else len(df)-1
                # 然后截取
                df_plot = df.iloc[start_i:end_i,:]
                df_plot.index=pd.to_datetime(df_plot['Date'])
                # 然后以这个绘图
                mydpi = 300
                fig1, _ = mpf.plot(df_plot,
                               type='candle',
                               style='yahoo',
                               figsize =(3240/mydpi,2520/mydpi), 
                               volume=True,
                               axisoff=True,
                               returnfig=True, 
                               scale_padding=0.2)
                # 这里生成文件名
                dest_path = os.path.join(dest_dir, "{}-{}-{}.jpg".format(i_img, df3.iat[0, id_index],df3.iat[0,Date_index]))
                i_img += 1
                fig1.savefig(dest_path,dpi=mydpi)
                plt.clf()
                plt.close()
                
                last_i = df3.iat[k, i_index]
                last_close= df3.iat[k, close_index]
                
                pass
            # 
            k += 1
        pbar.update(1)
        # print(list(df3['i']))
        # print(duo_days)
        # break



print("平均天数:{:.2f}, 平均涨幅:{:.2f}%, 平均胜率:{:.2f}%".format(
    sum(duo_days)/len(duo_days),
    sum(duo_rises)/len(duo_rises)*100,
    len([i for i in duo_rises if i > 0])/len(duo_rises)*100
))

  0%|          | 0/4276 [00:00<?, ?it/s]

In [ ]:
# 我这里看看某天数的数据
dict_duo = {}      # 涨幅信息
dict_duo_days = {} # 天数信息
for i in range(len(duo_days)):
    # 遍历
    if not duo_days[i] in dict_duo:
        # 如果没有，就新建
        dict_duo[duo_days[i]] = []
        dict_duo_days[duo_days[i]] = 0
    dict_duo[duo_days[i]].append(duo_rises[i]) # 添加涨幅
    dict_duo_days[duo_days[i]] += duo_days[i]  # 添加天数
# 这里统计一下涨幅和胜率
# 总的天数:
total_days = sum(duo_days)
for key in dict_duo:
    # 遍历输出
    _up = [i for i in dict_duo[key] if i > 0] # 筛选涨幅大于0的。
    print("{},\t数量:{}:\t占比:{:.2f}%\t平均涨幅:{:.2f}%, \t胜率:{:.2f}%".format(
        key,
        dict_duo_days[key],
        dict_duo_days[key]/total_days*100,
        sum(dict_duo[key])/len(dict_duo[key]),
        len(_up)/len(dict_duo[key])*100
    ))   

In [ ]:
dict_duo2 = {
    "天数":[],
    "数量":[],
    "占比":[],
    "平均涨幅":[],
    "胜率":[],
}

for key in dict_duo:
    _up = [i for i in dict_duo[key] if i > 0]
    dict_duo2['天数'].append(key)
    dict_duo2['数量'].append(dict_duo_days[key])
    dict_duo2['占比'].append(dict_duo_days[key]/total_days*100)
    dict_duo2['平均涨幅'].append(sum(dict_duo[key])/len(dict_duo[key]))
    dict_duo2['胜率'].append(len(_up)/len(dict_duo[key])*100)
# 这里保存成文件
dt_out = pd.DataFrame(dict_duo2)
dt_out

In [ ]:
dt_out.sort_values(by='天数',inplace=True)
dt_out.head(50)

可以看到至少要10天以后，胜率在7成左右，但10天以前的占比总和大概是48%了

In [ ]:
dt_out.to_excel("均线多头.xlsx")